In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:

import os
import glob
import numpy as np
import scipy.io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa
import pandas as pd
import matplotlib.pyplot as plt
adam = tf.keras.optimizers.Adam(learning_rate=0.0005)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Input, Conv1D, MaxPooling1D, Flatten, Dense
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Reshape
temperature = 0.03
learning_rate=0.001

def dataloader(path, featType):
    """
    Load data from a MATLAB file.

    Args:
        path (str): Path to the MATLAB file.
        featType (int): Type of features to load.

    Returns:
        Tuple: Tuple containing input features, labels, weights, and additional information.
    """
    data = scipy.io.loadmat(path)
    print(data.keys())

    AF = data['AF']
    x1 = AF[:-2]
    y = AF[-2]
    w = AF[-1]

    if featType == 1:
        x = x1
    else:
        x2 = data['CF']
        x = np.concatenate((x1, x2), axis=0)
    return x.T, y.T, w.T, data['CF_info']

def calculate_accuracy(arr1, arr2):
    """
    Calculate the accuracy between two arrays.

    Args:
        arr1 (array): First array.
        arr2 (array): Second array.

    Returns:
        float: Accuracy between the two arrays.
    """
    count = sum(1 for itr1, itr2 in zip(arr1, arr2) if itr1 == itr2)
    return count / len(arr1)

def normalization(feats):

    """
    Normalize the input features using standard scaling.

    Args:
        feats (array): Input features.

    Returns:
        array: Normalized features.
    """
    df = pd.DataFrame(feats)
    scaler = StandardScaler()
    x_new = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return x_new

def make_partitions(arr_words, arr_labels):

    """
    Create partitions based on word boundaries and labels.

    Args:
        arr_words (array): Array of words.
        arr_labels (array): Array of labels.

    Returns:
        array: Partitions based on word boundaries and labels.
    """
    v = []
    temp = []

    for i in range(len(arr_words) - 1):
        word = arr_words[i]
        next_word = arr_words[i + 1]
        temp.append(arr_labels[i])

        if word != next_word or i == len(arr_words) - 2:
            if i == len(arr_words) - 2:
                temp.append(arr_labels[i + 1])

            numpy_temp = np.array(temp)
            temp_max = np.amax(numpy_temp)
            numpy_temp = np.divide(numpy_temp, temp_max)
            v = np.concatenate((v, numpy_temp), axis=None)
            temp.clear()

    v1 = [1 if i == 1 else 0 for i in v]
    return v1
    
fatyp = 'TypicalFA_comb1'
drivepath = 'finalData/'+ fatyp +'/';
#featFiles = 'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress'               #glob.glob(drivepath + '*train*')
filee = drivepath+'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
featType = 1; #Acoustic or Acoustic+context
if featType == 1:
  original_dim = 19
else:
  original_dim = 38
# print('Classification with::::::',os.path.basename(filee))
train_path = filee; test_path = filee.replace('train','test')
# print('test file:::::::',os.path.basename(test_path))
xtrain, ytrain, wtrain, info_train = dataloader(train_path, featType); 
xtrain1, ytrain1, wtrain1, info_train1 = dataloader(train_path, featType=2);
xtest, ytest, wtest ,info_test = dataloader(test_path, featType)
xtest1, ytest1, wtest1, info_test1 = dataloader(test_path, featType=2);
xtest_a = normalization(xtest)
xtest_ac = normalization(xtest1)
xtrain = normalization(xtrain)
xtrain1 = normalization(xtrain1)

woPP=[]; wPP=[]
input_shape1 = (19,1)
input_shape2 = (38,1)
temperature = 0.03
learning_rate=0.001

2023-06-09 06:10:36.353404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/project/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-06-09 06:10:40.021458: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-06-09 06:1

dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
